# Image directory

In [ ]:
# download 40-50 different image of people from internet
# create different folder for each person
# download 40-50 images of the two people (1. whose video was used for deepfake and 2. whose image was swapped on the video)
# rename the downloaded images with the names of the people.
# use mtcnn for feature extraction of these images and store in a new folder

In [ ]:

!pip install patool
import patoolib
patoolib.extract_archive('dataset zip file location')

In [ ]:
!pip install mtcnn

In [ ]:
from mtcnn import MTCNN
from cv2 import imread
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:

##------------------------------------------------------> no need for this box
filename = 'file location of one image'
image = Image.open(filename)
# convert to RGB, if needed
image = image.convert('RGB')
# convert to array
pixels = np.asarray(image)
plt.imshow(pixels)
print(pixels.shape)

In [ ]:
# create the face detector, using default weights
detector = MTCNN()
# detect faces in the image
results = detector.detect_faces(pixels)

In [ ]:
## Assuming that only 1 face is present
# extract the bounding box from the first face
x1, y1, width, height = results[0]['box']

# bug fix - Sometimes the library will return a negative pixel index
x1, y1 = abs(x1), abs(y1)
x2, y2 = x1 + width, y1 + height

In [ ]:
# extract the face. this is like cropping the face from image
face = pixels[y1:y2, x1:x2]

In [ ]:

# resize pixels to the model size
import cv2 as cv
image = Image.fromarray(face)

face_array = np.asarray(image)
# Lets show the extracted face

image= cv.resize(face_array, dsize=(160,160), interpolation=cv.INTER_LINEAR)

plt.imshow(image)

In [ ]:
# Function to extract a single face from a given photograph
def extract_face(filename=None, image_pixels=None):
    if filename is not None:
        image = Image.open(filename)
        image = image.convert('RGB')
        pixels = np.asarray(image)
    elif image_pixels is not None:
        pixels = image_pixels
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    box_dimensions = (x1, y1, width, height)
    image = Image.fromarray(face)
    face_array = np.asarray(image)
    image= cv.resize(face_array, dsize=(160,160), interpolation=cv.INTER_LINEAR)


    return face_array, box_dimensions

In [ ]:
# load images and extract faces for all images in a directory
import cv2 as cv
def load_faces(directory,output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    faces = []
    # enumerate files
    for filename in os.listdir(directory):
        # path
        path = os.path.join(directory,filename)
        # get face
        face, _ = extract_face(filename=path)
        # store

        # Save the face to the output directory with a new name
        face_filename =  os.path.splitext(filename)[0] + "_face.jpg"
        output_path = os.path.join(output_dir, face_filename)
        cv.imwrite(output_path, face)
        faces.append(face)
    return faces

In [ ]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = [], []
    out_dir="output directory location"
    for subdir in os.listdir(directory):
        path = os.path.join(directory, subdir)
        if not os.path.isdir(path):
            continue
        faces = load_faces(path,out_dir)
        labels = [subdir for _ in range(len(faces))]
        print(f'Loaded {len(faces)} examples for class: {subdir}')
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

In [ ]:
%%time
# load train dataset
#../input/5-celebrity-faces-dataset/train/ben_afflek/httpcsvkmeuaeccjpg.jpg
X_train, y_train = load_dataset(r'dataset location')
print(X_train.shape, y_train.shape)
